In [ ]:
from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

# Create list of file name vs time stamp 

To do so, run the following notebook -> [create_list_of_file_name_vs_time_stamp](create_list_of_file_name_vs_time_stamp.ipynb)

# Sort files according to T and P parameters

In [ ]:
import ipywe.fileselector
import pandas as pd
import numpy as np

import os, re
from collections import OrderedDict

In [ ]:
file_name_ui = ipywe.fileselector.FileSelectorPanel(instruction='Select file created by the create_list_of_file_name_vs_time_stamp',
                                                   start_dir = system.System.get_working_dir())
file_name_ui.show()

In [ ]:
list_of_filename_vs_timestamp = file_name_ui.selected
pd_file_vs_timestamp = pd.read_csv(list_of_filename_vs_timestamp)
pd_file_vs_timestamp.columns = ['filename','time_stamp_s','time_human_readable','time_offset_s']
list_of_files = pd_file_vs_timestamp['filename']
pd_file_vs_timestamp.head()

### Parse the file names

In [ ]:


my_re = r"\w*_(?P<temperature>\w+)_(?P<pressure>\w+)_\d{4}_(?P<digit>\d{4}).tiff$"
exp_dict = OrderedDict()
_new_exp = {'list_of_files': [], 
            'T':'', 
            'P':'', 
            'list_of_images':[], 
            'working_image': {'type': '',
                              'image': []},
            'folder': ''}

previous_T = ''
previous_P = ''

exp_index = 0
for _index, _file in enumerate(list_of_files):
    _path = os.path.dirname(_file)
    _file = os.path.basename(_file)
    m = re.match(my_re, _file)
    if m is None:
        continue

    if m:
        _T = m.group('temperature')
        _P = m.group('pressure')
    else:
        _T = 'N/A'
        _P = 'N/A'

    # first entry
    if _new_exp['list_of_files'] == []:
        _new_exp['list_of_files'] = [os.path.basename(_file)]
        _new_exp['folder'] = _path
        
        _new_exp['T'] = _T
        previous_T = _T
            
        _new_exp['P'] = _P
        previous_P = _P
            
    # after first entry
    else:
        
        if (_T == previous_T) and (_P == previous_P):
            # found the same T and P, just append file to list of files
            _new_exp['list_of_files'].append(_file)
            
        else:
            # we found a new T or new P, save previous dict and initialization of new one
            exp_dict[str(exp_index)] = _new_exp
            exp_index += 1
            _new_exp = {'list_of_files': [], 
                        'T':'', 
                        'P':'', 
                        'list_of_images':[], 
                        'working_image': {'type': '',
                                          'image': []},
                        'folder': ''}

            # start recording new entry
            _new_exp['list_of_files'] = [os.path.basename(_file)]
            _new_exp['folder'] = _path
        
            _new_exp['T'] = _T
            previous_T = _T
            
            _new_exp['P'] = _P
            previous_P = _P
        
    # last entry, force the save of the dictionary
    if _index == len(list_of_files)-1:
        exp_dict[str(exp_index)] = _new_exp
        
#exp_dict

History

In [ ]:
for _key in exp_dict.keys():
    _item = exp_dict[_key]
    _t = _item['T']
    _p = _item['P']
    _nbr_files = len(_item['list_of_files'])
    
    print("T:{} and P:{} with {} files".format(_t, _p, _nbr_files))

# Load the data 

In [ ]:
from NeuNorm.normalization import Normalization
from ipywidgets import widgets
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

In [ ]:
w = widgets.IntProgress(description="Full Progress")
w.max = len(exp_dict.keys())
display(w)

for _index, _key in enumerate(exp_dict.keys()):
    _item = exp_dict[_key]
    _path = _item['folder']
    list_files = _item['list_of_files']
    full_list_files = [os.path.join(_path, _file) for _file in list_files]
    o_norm = Normalization()
    o_norm.load(file=full_list_files, notebook=True)
    _data = o_norm.data['sample']['data']
    _item['list_of_images'] = _data
    exp_dict[_key] = _item
    
    w.value = _index+1
    
w.close()

# Which image to use for each group 

Should we use for each group:
* the last image
or
* the mean of all images

In [ ]:
# #working with group = 0 and 1

# group = exp_dict['0']
# nbr_images_in_group = len(group['list_of_images'])
# mean_group_0 = []
# #median_group_0 = []
# std_group_0 = []

# for _image in group['list_of_images']:
# #     _median_value = np.median(_image)
# #     median_group_0.append(_median_value)

#     _mean_value = np.mean(_image)
#     mean_group_0.append(_mean_value)

#     _std_value = np.std(_image)
#     std_group_0.append(_std_value)

# ###
# group = exp_dict['1']
# nbr_images_in_group = len(group['list_of_images'])
# mean_group_1 = []
# # median_group_1 = []
# std_group_1 = []

# for _image in group['list_of_images']:
# #     _median_value = np.median(_image)
# #     median_group_1.append(_median_value)

#     _mean_value = np.mean(_image)
#     mean_group_1.append(_mean_value)

#     _std_value = np.std(_image)
#     std_group_1.append(_std_value)
    
# plt.figure(0)
# plt.plot(std_group_1, 'r')   

In [ ]:
def calculate_working_image(list_of_images):
    """Determine here if we should use the last image or a mean of all the images"""
    # TO DO
    _type = 'last_image'
    _type = 'mean'
    _image = np.mean(list_of_images, axis=0)
    
    working = {'type': _type,
               'image': _image}
    
    return working    

In [ ]:
for _group_number in exp_dict.keys():
    _group = exp_dict[_group_number]
    nbr_images = len(_group['list_of_images'])
    if nbr_images == 1:
        _group['working_image']['type'] = 'last_image'
        _group['working_image']['image'] = _group['list_of_images'][0].copy()
    else:
        _list_images = _group['list_of_images']
        
        _result_working_image = calculate_working_image(_list_images)
        _group['working_image'] = _result_working_image
    exp_dict[_group_number] = _group
            

# Display images 

In [ ]:
from collections import OrderedDict
import numpy as np

In [ ]:
temp_exp_dict = OrderedDict()
_data = np.random.rand(20,20)
temp_exp_dict['0'] = {'P': '00Bar', 'T': 'cooling', 'working_image': {'image': _data}}
_data = np.random.rand(20,20)*2
temp_exp_dict['1'] = {'P': '06Bar', 'T': 'cooling', 'working_image': {'image': _data}}
_data = np.random.rand(20,20)*3
temp_exp_dict['2'] = {'P': '08Bar', 'T': 'cooling', 'working_image': {'image': _data}}
_data = np.random.rand(20,20)*4
temp_exp_dict['3'] = {'P': '10Bar', 'T': 'cooling', 'working_image': {'image': _data}}
_data = np.random.rand(20,20)*5
temp_exp_dict['4'] = {'P': '10Bar', 'T': 'cooling', 'working_image': {'image': _data}}
_data = np.random.rand(20,20)*100
temp_exp_dict['5'] = {'P': '10Bar', 'T': 'cooling', 'working_image': {'image': _data}}
_data = np.random.rand(20,20)*200
temp_exp_dict['6'] = {'P': '10Bar', 'T': 'cooling', 'working_image': {'image': _data}}
_data = np.random.rand(20,20)*300
temp_exp_dict['7'] = {'P': '10Bar', 'T': 'cooling', 'working_image': {'image': _data}}
_data = np.random.rand(20,20)*0.5
temp_exp_dict['8'] = {'P': '10Bar', 'T': 'cooling', 'working_image': {'image': _data}}

In [ ]:
%gui qt

In [ ]:
from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_file_metadata_display.ui')

from __code.file_metadata_display import Interface

In [ ]:
o_gui = Interface(exp_dict=exp_dict)
o_gui.show()

In [ ]:
ex